# US Golf Course Information Kiosk

Detailed information on golf courses with green fees range and tees, this complete golf courses database has 15,606 records across 7,891 cities over 53 states in the United States. Each record is comprised of address, street, phone number, zip code, hole, architect, year built, public/private, guest policy, credit card, golf season, range, rental club, pro in house, metal spikes okay, weekday, weekend, tee time welcomed, rental cart available, championship par/yards/slope/USGA, middle par/yards/slope/USGA and forward par/yards/slope/USGA

https://www.usabledatabases.com/database/golf-courses-in-us/

## Step 1:  Extract, clean and transform the input dataset

Before running this extract, clean and transform process, make sure that you update the config.py file with a Google Map API Key as gkey 


In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from config import gkey

# CSV files
course = "data/course.csv"
city = "data/city.csv"
state = "data/state.csv"

### Longitude and Latitude for mapping golf courses using street address

In [2]:
# Create the data frame - golf courses
course_df = pd.read_csv(course, 
                        delimiter=',', 
                        skipinitialspace=True, 
                        dtype={"zip_code":object, "hole":object })

# Add columns for longitude and latitude
course_df["lng"] = ""
course_df["lat"] = ""

golf_courses = course_df.rename(columns={"id": "course_id",
                                         "title": "course"})

# Replace &amp; with &
golf_courses = golf_courses.replace('&amp;','&', regex=True)

golf_courses

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,lng,lat
0,1,1,Albertville Golf & Country Club,Country Club Rd,"Albertville, Alabama 35950",35950,(256) 878-4403,18,Leon Howard,1966,...,72,6068,117,68.00,72,5196,NaN,NaN,,
1,2,1,A. J. Jolly Golf Course,5350 S US Hwy 27,"Alexandria, Kentucky 41001",41001,(606) 635-2106,18,NaN,1962,...,71,5942,115,67.60,75,5418,118,70.30,,
2,3,2,Willow Point Country Club,3054 Willow Point Rd,"Alexander City, Alabama 35010",35010,(256) 234-2572,18,Thomas H. Nicol,1961,...,72,6631,135,72.20,73,5373,120,70.70,,
3,4,3,Alpine Bay Resort,9855 Renfroe Rd,"Alpine, Alabama 35014",35014,(256) 268-2920,18,"Robert Trent Jones, Sr.",1972,...,72,6518,126,69.90,72,5518,120,69.80,,
4,5,4,Maple Hills Golf,Hwy 75,"Altoona, Alabama 35952",35952,(205) 466-7600,9,Bill Ellison,1975,...,31,1800,NaN,NaN,31,1800,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15601,15750,8219,Legion Town & Country Club,141 Airport Rd,"Thermopolis, Wyoming 82443",82443,(307) 864-5294,9,Don Brunk,1962,...,35,2966,111,33.90,37,2836,111,35.00,,
15602,15751,8220,Cottonwood Country Club,15th St,"Torrington, Wyoming 82240",82240,(307) 532-3868,18,NaN,NaN,...,72,6298,112,67.70,73,5344,112,69.20,,
15603,15752,8221,Cedar Pines Golf Course,2579 N Hwy 116,"Upton, Wyoming 82730",82730,(307) 468-2847,9,NaN,1984,...,36,3198,NaN,NaN,38,2521,NaN,NaN,,
15604,15753,8222,Wheatland Golf Club,1253 E Cole,"Wheatland, Wyoming 82201",82201,(307) 322-3675,9,NaN,1960,...,36,3064,119,33.95,36,2557,116,34.25,,


In [3]:
# Exclude all golf courses that do not have championship, middle, and forward tee information
nan_value = float("NaN")
golf_courses.replace("", nan_value, inplace=True)
golf_courses.dropna(subset = ["championship_par"], inplace=True)
golf_courses.dropna(subset = ["middle_par"], inplace=True)
golf_courses.dropna(subset = ["forward_par"], inplace=True)
golf_courses.dropna(subset = ["championship_slope"], inplace=True)
golf_courses.dropna(subset = ["middle_slope"], inplace=True)
golf_courses.dropna(subset = ["forward_slope"], inplace=True)
golf_courses.dropna(subset = ["championship_usga"], inplace=True)
golf_courses.dropna(subset = ["middle_usga"], inplace=True)
golf_courses.dropna(subset = ["forward_usga"], inplace=True)

# Exclude all golf courses that do not have hole, public-private, season information
golf_courses.dropna(subset = ["hole"], inplace=True)
golf_courses.dropna(subset = ["public_private"], inplace=True)
golf_courses.dropna(subset = ["golf_season"], inplace=True)

golf_courses

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,lng,lat
1,2,1,A. J. Jolly Golf Course,5350 S US Hwy 27,"Alexandria, Kentucky 41001",41001,(606) 635-2106,18,NaN,1962,...,71,5942,115,67.60,75,5418,118,70.30,NaN,NaN
2,3,2,Willow Point Country Club,3054 Willow Point Rd,"Alexander City, Alabama 35010",35010,(256) 234-2572,18,Thomas H. Nicol,1961,...,72,6631,135,72.20,73,5373,120,70.70,NaN,NaN
3,4,3,Alpine Bay Resort,9855 Renfroe Rd,"Alpine, Alabama 35014",35014,(256) 268-2920,18,"Robert Trent Jones, Sr.",1972,...,72,6518,126,69.90,72,5518,120,69.80,NaN,NaN
5,6,5,Andalusia Country Club,210 Country Club Dr,"Andalusia, Alabama 36420",36420,(334) 222-5282,18,Evans Barnes,1927,...,72,6151,124,69.00,72,5563,119,71.20,NaN,NaN
8,9,7,Anniston Country Club,601 Highland Ave,"Anniston, Alabama 36207",36207,(256) 237-4615,18,NaN,1909,...,70,5924,118,68.70,72,5086,109,68.50,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15591,15740,8213,White Mountain Golf Course,1501 Clubhouse Dr,"Rock Springs, Wyoming 82901",82901,(307) 352-1415,18,Lynn Radike,1979,...,72,6741,118,71.00,73,5616,115,73.10,NaN,NaN
15592,15741,8214,Old Baldy Club,E PicPike Rd,"Saratoga, Wyoming 82331",82331,(307) 326-5222,18,Henry Hughes,1962,...,72,6649,114,69.80,72,5410,110,67.60,NaN,NaN
15594,15743,8215,"Powder Horn Ranch & Golf Club, The",14 Clubhouse Rd,"Sheridan, Wyoming 82801",82801,(307) 672-5323,18,Dick Bailey,1997,...,72,6469,122,70.00,72,5878,122,71.90,NaN,NaN
15595,15744,8215,Kendrick Golf Course,Big Goose Rd,"Sheridan, Wyoming 82801",82801,(307) 674-8148,18,Edward A. Hunnicutt,1932,...,72,6404,111,69.10,72,5180,111,68.00,NaN,NaN


### Checking for duplicate rows of golf courses, cities, states

In [4]:
# Checking for duplicate rows in golf courses
duplicateRow = golf_courses[golf_courses.duplicated()]
duplicateRow

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,lng,lat


In [5]:
# Create a subset data frame - MN golf courses
subset_df = golf_courses[((golf_courses["zip_code"] >= "56001") & (golf_courses["zip_code"] <= "56007"))]

subset_df

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,lng,lat
7589,7676,3616,Green Lea Golf Course,101 E Richway Dr,"Albert Lea, Minnesota 56007",56007,(507) 373-1061,18,NaN,NaN,...,72,5970,120,69.30,72,5404,126,71.40,NaN,NaN
7846,7937,3776,Mankato Golf Club,Golf Club Rd,"Mankato, Minnesota 56002",56002,(507) 387-5636,18,William Langford & Theodore J. Moreau,1925,...,71,5990,128,69.20,71,5009,126,68.90,NaN,NaN
7848,7939,3776,Regulation at Terrace View Golf Course,Hwy 22 S,"Mankato, Minnesota 56001",56001,(507) 625-7665,9,NaN,1973,...,36,3093,114,34.50,36,2464,110,34.00,NaN,NaN
7909,8001,3810,North Links Golf Course,RFD 2,"North Mankato, Minnesota 56003",56003,(507) 947-3355,18,Patrick Wyss,1991,...,72,5624,113,67.00,72,4659,114,67.00,NaN,NaN


In [6]:
# Create a subset data frame - MN golf courses
subset_df = golf_courses[((golf_courses["zip_code"] >= "55001") & (golf_courses["zip_code"] <= "56763"))]

subset_df

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,lng,lat
7585,7672,3613,Cedar River Country Club,Hwy 56 W,"Adams, Minnesota 55909",55909,(507) 582-3595,18,NaN,1965,...,72,6076,117,68.40,74,5553,119,71.30,NaN,NaN
7586,7673,3614,Adrian Golf Course,Hwy 91 S,"Adrian, Minnesota 56110",56110,(507) 483-2722,9,Drew Demarest,1994,...,36,3140,120,35.00,36,2601,115,34.80,NaN,NaN
7587,7674,3615,Albany Golf Club,500 Church St,"Albany, Minnesota 56307",56307,(320) 845-2505,18,Willie Kidd,1960,...,72,6023,119,68.20,74,5268,117,69.70,NaN,NaN
7589,7676,3616,Green Lea Golf Course,101 E Richway Dr,"Albert Lea, Minnesota 56007",56007,(507) 373-1061,18,NaN,NaN,...,72,5970,120,69.30,72,5404,126,71.40,NaN,NaN
7594,7681,3619,Southbrook Golf & Country Club,511 Morrison Ave,"Annandale, Minnesota 55302",55302,(320) 274-2341,18,Andy Lindquist,1997,...,72,6120,116,68.70,72,5137,112,69.10,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8061,8153,3899,Prestwick Golf Club at Wedgewood,9555 Wedgewood Dr,"Woodbury, Minnesota 55125",55125,(651) 731-4779,18,THK Associates,1986,...,72,6334,123,70.70,72,5267,121,71.00,NaN,NaN
8063,8155,3900,Prairie View Golf Links,Hwy 266 No,"Worthington, Minnesota 56187",56187,(507) 372-8670,18,Joel Goldstrand,1983,...,71,5989,111,68.30,71,5105,111,69.00,NaN,NaN
8064,8156,3900,Worthington Country Club,1414 Liberty Dr,"Worthington, Minnesota 56187",56187,(507) 376-4281,18,Leo Johnson,1919,...,71,5982,123,68.60,71,5243,122,70.80,NaN,NaN
8065,8157,3901,"Greenwood Golf Links, Inc.",4520 E Viking Blvd,"Wyoming, Minnesota 55092",55092,(651) 462-4653,18,C. M. Johnson,1985,...,72,5438,103,66.50,72,4791,112,67.30,NaN,NaN


In [7]:
# Create a subset data frame - variety of golf courses
bool_series = golf_courses["zip_code"].str.startswith("010", na = False) 
df1 = golf_courses[bool_series]
bool_series = golf_courses["zip_code"].str.startswith("6000", na = False) 
df2 = golf_courses[bool_series]
bool_series = golf_courses["zip_code"].str.startswith("5600", na = False) 
df3 = golf_courses[bool_series]
bool_series = golf_courses["zip_code"].str.startswith("920", na = False) 
df4 = golf_courses[bool_series]
bool_series = golf_courses["zip_code"].str.startswith("2001", na = False) 
df5 = golf_courses[bool_series]

subset_df = df1.append(df2, ignore_index = True)
subset_df = subset_df.append(df3, ignore_index = True)
subset_df = subset_df.append(df4, ignore_index = True)
subset_df = subset_df.append(df5, ignore_index = True)
subset_df

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,lng,lat
0,6390,2981,Crestview Country Club,Shoemaker Ln,"Agawam, Massachusetts 01001",01001,(413) 786-0917,18,Geoffrey Cornish,1957,...,72,6695,124,72.30,73,5571,110,66.70,NaN,NaN
1,6393,2983,Hickory Ridge Country Club,191 W Pomeroy Ln,"Amherst, Massachusetts 01002",01002,(413) 253-9320,18,William G. Robinson,1973,...,74,6427,128,71.10,74,5340,117,70.40,NaN,NaN
2,6445,3013,Chicopee Country Club,1290 Burnett Rd,"Chicopee, Massachusetts 01020",01020,(413) 594-9295,18,Geoffrey Cornish,1963,...,71,6109,120,70.40,71,5123,115,72.40,NaN,NaN
3,6466,3028,Elmcrest Country Club,105 Somersville Rd,"East Longmeadow, Massachusetts 01028",01028,(413) 525-8444,18,NaN,1967,...,70,5904,115,68.10,72,5363,119,70.90,NaN,NaN
4,6474,3034,Oak Ridge Golf Club,850 S Westfield St,"Feeding Hills, Massachusetts 01030",01030,(413) 789-7307,18,George & Tom Fazio,1974,...,70,6390,120,70.00,70,5297,124,70.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,1632,692,Carlton Oaks Lodge & Country Club,9200 Inwood Dr,"Santee, California 92071",92071,(619) 448-8500,18,William W. Mast,1950,...,72,6537,127,71.30,72,4537,114,67.10,NaN,NaN
58,1647,702,Lomas Santa Fe Country Club,1505 Lomas Santa Fe Dr,"Solana Beach, California 92075",92075,(858) 755-1547,18,William F. Bell,1964,...,72,6208,122,70.00,72,5830,138,76.10,NaN,NaN
59,1756,755,Shadowridge Golf Club,1980 Gateway Dr,"Vista, California 92083",92083,(760) 727-7706,18,David Rainville,1978,...,72,6361,114,69.90,72,5693,135,74.90,NaN,NaN
60,1757,755,Vista Valley Country Club,29354 Vista Valley Dr,"Vista, California 92084",92084,(760) 758-5275,18,Ted Robinson,1979,...,71,6035,125,69.60,71,5627,133,74.40,NaN,NaN


In [8]:
# Create a subset data frame - All golf courses
subset_df = golf_courses

subset_df

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,lng,lat
1,2,1,A. J. Jolly Golf Course,5350 S US Hwy 27,"Alexandria, Kentucky 41001",41001,(606) 635-2106,18,NaN,1962,...,71,5942,115,67.60,75,5418,118,70.30,NaN,NaN
2,3,2,Willow Point Country Club,3054 Willow Point Rd,"Alexander City, Alabama 35010",35010,(256) 234-2572,18,Thomas H. Nicol,1961,...,72,6631,135,72.20,73,5373,120,70.70,NaN,NaN
3,4,3,Alpine Bay Resort,9855 Renfroe Rd,"Alpine, Alabama 35014",35014,(256) 268-2920,18,"Robert Trent Jones, Sr.",1972,...,72,6518,126,69.90,72,5518,120,69.80,NaN,NaN
5,6,5,Andalusia Country Club,210 Country Club Dr,"Andalusia, Alabama 36420",36420,(334) 222-5282,18,Evans Barnes,1927,...,72,6151,124,69.00,72,5563,119,71.20,NaN,NaN
8,9,7,Anniston Country Club,601 Highland Ave,"Anniston, Alabama 36207",36207,(256) 237-4615,18,NaN,1909,...,70,5924,118,68.70,72,5086,109,68.50,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15591,15740,8213,White Mountain Golf Course,1501 Clubhouse Dr,"Rock Springs, Wyoming 82901",82901,(307) 352-1415,18,Lynn Radike,1979,...,72,6741,118,71.00,73,5616,115,73.10,NaN,NaN
15592,15741,8214,Old Baldy Club,E PicPike Rd,"Saratoga, Wyoming 82331",82331,(307) 326-5222,18,Henry Hughes,1962,...,72,6649,114,69.80,72,5410,110,67.60,NaN,NaN
15594,15743,8215,"Powder Horn Ranch & Golf Club, The",14 Clubhouse Rd,"Sheridan, Wyoming 82801",82801,(307) 672-5323,18,Dick Bailey,1997,...,72,6469,122,70.00,72,5878,122,71.90,NaN,NaN
15595,15744,8215,Kendrick Golf Course,Big Goose Rd,"Sheridan, Wyoming 82801",82801,(307) 674-8148,18,Edward A. Hunnicutt,1932,...,72,6404,111,69.10,72,5180,111,68.00,NaN,NaN


In [9]:
# Create the data frame - golf cities
city_df = pd.read_csv(city, delimiter=',', skipinitialspace=True)

golf_cities = city_df.rename(columns={"id": "city_id", 
                                      "title": "city_title", 
                                      "count": "city_count", 
                                      "slug": "city_slug"})
golf_cities

,city_id,state_id,city_title,city_count,city_slug
0,1,1,Albertville,2,albertville
1,2,1,Alexander City,1,alexandercity
2,3,1,Alpine,1,alpine
3,4,1,Altoona,1,altoona
4,5,1,Andalusia,2,andalusia
...,...,...,...,...,...
7886,8219,53,Thermopolis,1,thermopolis
7887,8220,53,Torrington,1,torrington
7888,8221,53,Upton,1,upton
7889,8222,53,Wheatland,1,wheatland


In [10]:
# Checking for duplicate rows in golf cities
duplicateRow = golf_cities[golf_cities.duplicated()]
duplicateRow

,city_id,state_id,city_title,city_count,city_slug


In [11]:
# Create the data frame - golf states
state_df = pd.read_csv(state, delimiter=',', skipinitialspace=True)

golf_states = state_df.rename(columns={"id": "state_id", 
                                       "title": "state", 
                                       "count": "state_count", 
                                       "slug": "state_slug"})
golf_states

,state_id,state,state_count,state_slug
0,1,Alabama,272,alabama
1,2,Alaska,16,alaska
2,3,Arizona,308,arizona
3,4,Arkansas,179,arkansas
4,5,California,976,california
5,6,Colorado,223,colorado
6,7,Connecticut,177,connecticut
7,8,Delaware,39,delaware
8,9,District of Columbia,6,districtofcolumbia
9,10,Florida,1094,florida


In [12]:
# Checking for duplicate rows in golf states
duplicateRow = golf_states[golf_states.duplicated()]
duplicateRow

,state_id,state,state_count,state_slug


### Merge the data frames to create one complete dataset

In [13]:
# Merge the city and state dataframes using a left join
city_state_df = pd.merge(golf_cities, golf_states, on="state_id", how="left")
city_state_df

,city_id,state_id,city_title,city_count,city_slug,state,state_count,state_slug
0,1,1,Albertville,2,albertville,Alabama,272,alabama
1,2,1,Alexander City,1,alexandercity,Alabama,272,alabama
2,3,1,Alpine,1,alpine,Alabama,272,alabama
3,4,1,Altoona,1,altoona,Alabama,272,alabama
4,5,1,Andalusia,2,andalusia,Alabama,272,alabama
...,...,...,...,...,...,...,...,...
7886,8219,53,Thermopolis,1,thermopolis,Wyoming,48,wyoming
7887,8220,53,Torrington,1,torrington,Wyoming,48,wyoming
7888,8221,53,Upton,1,upton,Wyoming,48,wyoming
7889,8222,53,Wheatland,1,wheatland,Wyoming,48,wyoming


In [14]:
# Merge the city_state dataframe with the golf course dataframe using a left join
golf_citystate = pd.merge(subset_df, city_state_df, on="city_id" ,how="left")
golf_citystate

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,forward_usga,lng,lat,state_id,city_title,city_count,city_slug,state,state_count,state_slug
0,2,1,A. J. Jolly Golf Course,5350 S US Hwy 27,"Alexandria, Kentucky 41001",41001,(606) 635-2106,18,NaN,1962,...,70.30,NaN,NaN,1,Albertville,2,albertville,Alabama,272,alabama
1,3,2,Willow Point Country Club,3054 Willow Point Rd,"Alexander City, Alabama 35010",35010,(256) 234-2572,18,Thomas H. Nicol,1961,...,70.70,NaN,NaN,1,Alexander City,1,alexandercity,Alabama,272,alabama
2,4,3,Alpine Bay Resort,9855 Renfroe Rd,"Alpine, Alabama 35014",35014,(256) 268-2920,18,"Robert Trent Jones, Sr.",1972,...,69.80,NaN,NaN,1,Alpine,1,alpine,Alabama,272,alabama
3,6,5,Andalusia Country Club,210 Country Club Dr,"Andalusia, Alabama 36420",36420,(334) 222-5282,18,Evans Barnes,1927,...,71.20,NaN,NaN,1,Andalusia,2,andalusia,Alabama,272,alabama
4,9,7,Anniston Country Club,601 Highland Ave,"Anniston, Alabama 36207",36207,(256) 237-4615,18,NaN,1909,...,68.50,NaN,NaN,1,Anniston,4,anniston,Alabama,272,alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9079,15740,8213,White Mountain Golf Course,1501 Clubhouse Dr,"Rock Springs, Wyoming 82901",82901,(307) 352-1415,18,Lynn Radike,1979,...,73.10,NaN,NaN,53,Rock Springs,1,rocksprings,Wyoming,48,wyoming
9080,15741,8214,Old Baldy Club,E PicPike Rd,"Saratoga, Wyoming 82331",82331,(307) 326-5222,18,Henry Hughes,1962,...,67.60,NaN,NaN,53,Saratoga,2,saratoga,Wyoming,48,wyoming
9081,15743,8215,"Powder Horn Ranch & Golf Club, The",14 Clubhouse Rd,"Sheridan, Wyoming 82801",82801,(307) 672-5323,18,Dick Bailey,1997,...,71.90,NaN,NaN,53,Sheridan,3,sheridan,Wyoming,48,wyoming
9082,15744,8215,Kendrick Golf Course,Big Goose Rd,"Sheridan, Wyoming 82801",82801,(307) 674-8148,18,Edward A. Hunnicutt,1932,...,68.00,NaN,NaN,53,Sheridan,3,sheridan,Wyoming,48,wyoming


In [15]:
regex = r'(?P<c_city>[^,]+)\s*,\s*'
golf_address = golf_citystate.join(golf_citystate["address"].str.extract(regex))
golf_address

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,lng,lat,state_id,city_title,city_count,city_slug,state,state_count,state_slug,c_city
0,2,1,A. J. Jolly Golf Course,5350 S US Hwy 27,"Alexandria, Kentucky 41001",41001,(606) 635-2106,18,NaN,1962,...,NaN,NaN,1,Albertville,2,albertville,Alabama,272,alabama,Alexandria
1,3,2,Willow Point Country Club,3054 Willow Point Rd,"Alexander City, Alabama 35010",35010,(256) 234-2572,18,Thomas H. Nicol,1961,...,NaN,NaN,1,Alexander City,1,alexandercity,Alabama,272,alabama,Alexander City
2,4,3,Alpine Bay Resort,9855 Renfroe Rd,"Alpine, Alabama 35014",35014,(256) 268-2920,18,"Robert Trent Jones, Sr.",1972,...,NaN,NaN,1,Alpine,1,alpine,Alabama,272,alabama,Alpine
3,6,5,Andalusia Country Club,210 Country Club Dr,"Andalusia, Alabama 36420",36420,(334) 222-5282,18,Evans Barnes,1927,...,NaN,NaN,1,Andalusia,2,andalusia,Alabama,272,alabama,Andalusia
4,9,7,Anniston Country Club,601 Highland Ave,"Anniston, Alabama 36207",36207,(256) 237-4615,18,NaN,1909,...,NaN,NaN,1,Anniston,4,anniston,Alabama,272,alabama,Anniston
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9079,15740,8213,White Mountain Golf Course,1501 Clubhouse Dr,"Rock Springs, Wyoming 82901",82901,(307) 352-1415,18,Lynn Radike,1979,...,NaN,NaN,53,Rock Springs,1,rocksprings,Wyoming,48,wyoming,Rock Springs
9080,15741,8214,Old Baldy Club,E PicPike Rd,"Saratoga, Wyoming 82331",82331,(307) 326-5222,18,Henry Hughes,1962,...,NaN,NaN,53,Saratoga,2,saratoga,Wyoming,48,wyoming,Saratoga
9081,15743,8215,"Powder Horn Ranch & Golf Club, The",14 Clubhouse Rd,"Sheridan, Wyoming 82801",82801,(307) 672-5323,18,Dick Bailey,1997,...,NaN,NaN,53,Sheridan,3,sheridan,Wyoming,48,wyoming,Sheridan
9082,15744,8215,Kendrick Golf Course,Big Goose Rd,"Sheridan, Wyoming 82801",82801,(307) 674-8148,18,Edward A. Hunnicutt,1932,...,NaN,NaN,53,Sheridan,3,sheridan,Wyoming,48,wyoming,Sheridan


In [16]:
# Finding the best city to use
golf_address["city"] = golf_address.c_city.combine_first(golf_address.city_title)

golf_df = golf_address
golf_df

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,lat,state_id,city_title,city_count,city_slug,state,state_count,state_slug,c_city,city
0,2,1,A. J. Jolly Golf Course,5350 S US Hwy 27,"Alexandria, Kentucky 41001",41001,(606) 635-2106,18,NaN,1962,...,NaN,1,Albertville,2,albertville,Alabama,272,alabama,Alexandria,Alexandria
1,3,2,Willow Point Country Club,3054 Willow Point Rd,"Alexander City, Alabama 35010",35010,(256) 234-2572,18,Thomas H. Nicol,1961,...,NaN,1,Alexander City,1,alexandercity,Alabama,272,alabama,Alexander City,Alexander City
2,4,3,Alpine Bay Resort,9855 Renfroe Rd,"Alpine, Alabama 35014",35014,(256) 268-2920,18,"Robert Trent Jones, Sr.",1972,...,NaN,1,Alpine,1,alpine,Alabama,272,alabama,Alpine,Alpine
3,6,5,Andalusia Country Club,210 Country Club Dr,"Andalusia, Alabama 36420",36420,(334) 222-5282,18,Evans Barnes,1927,...,NaN,1,Andalusia,2,andalusia,Alabama,272,alabama,Andalusia,Andalusia
4,9,7,Anniston Country Club,601 Highland Ave,"Anniston, Alabama 36207",36207,(256) 237-4615,18,NaN,1909,...,NaN,1,Anniston,4,anniston,Alabama,272,alabama,Anniston,Anniston
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9079,15740,8213,White Mountain Golf Course,1501 Clubhouse Dr,"Rock Springs, Wyoming 82901",82901,(307) 352-1415,18,Lynn Radike,1979,...,NaN,53,Rock Springs,1,rocksprings,Wyoming,48,wyoming,Rock Springs,Rock Springs
9080,15741,8214,Old Baldy Club,E PicPike Rd,"Saratoga, Wyoming 82331",82331,(307) 326-5222,18,Henry Hughes,1962,...,NaN,53,Saratoga,2,saratoga,Wyoming,48,wyoming,Saratoga,Saratoga
9081,15743,8215,"Powder Horn Ranch & Golf Club, The",14 Clubhouse Rd,"Sheridan, Wyoming 82801",82801,(307) 672-5323,18,Dick Bailey,1997,...,NaN,53,Sheridan,3,sheridan,Wyoming,48,wyoming,Sheridan,Sheridan
9082,15744,8215,Kendrick Golf Course,Big Goose Rd,"Sheridan, Wyoming 82801",82801,(307) 674-8148,18,Edward A. Hunnicutt,1932,...,NaN,53,Sheridan,3,sheridan,Wyoming,48,wyoming,Sheridan,Sheridan


### All golf courses along the Mississippi River

In [17]:
# Golf courses along the Mississippi River - 
# Minnesota, Wisconsin, Iowa, Illinois, Missouri, Kentucky, Tennessee, Arkansas, Mississippi, Louisiana
mask = golf_df["state_id"].isin([24, 52, 16, 14, 26, 18, 44, 4, 25, 19])
MissRiver_golf = golf_df[mask]
MissRiver_golf

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,lat,state_id,city_title,city_count,city_slug,state,state_count,state_slug,c_city,city
386,607,227,Eagle Crest,"3926 Golf Course Drive,","Alma, AR 72921",72921,(479) 632-8857,18,Mark Hayes,1996,...,NaN,4,Alma,2,alma,Arkansas,179,arkansas,Alma,Alma
387,611,229,Millwood Landing Golf & RV Resort,596 Hwy 317,"Ashdown, Arkansas 71822",71822,(870) 898-6674,18,NaN,1975,...,NaN,4,Ashdown,2,ashdown,Arkansas,179,arkansas,Ashdown,Ashdown
388,616,233,Country Club at Bella Vista Village Golf Courses,101 Town Center,"Bella Vista, Arkansas 72714",72714,(501) 855-8172,18,Joe Finger,1968,...,NaN,4,Bella Vista,7,bellavista,Arkansas,179,arkansas,Bella Vista,Bella Vista
389,617,233,Highlands at Bella Vista Village Golf Courses,101 Town Center,"Bella Vista, Arkansas 72714",72714,(501) 855-8172,18,"Ault, Clark & Associates",1988,...,NaN,4,Bella Vista,7,bellavista,Arkansas,179,arkansas,Bella Vista,Bella Vista
390,618,233,Kingswood at Bella Vista Village Golf Courses,101 Town Center,"Bella Vista, Arkansas 72714",72714,(501) 855-8172,18,"Ault, Clark & Associates",1971,...,NaN,4,Bella Vista,7,bellavista,Arkansas,179,arkansas,Bella Vista,Bella Vista
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9057,15694,8184,Lake Breeze Golf Club,6333 State Road 110,"Winneconne, Wisconsin 54986",54986,(920) 582-7585,18,Homer Fieldhouse,1990,...,NaN,52,Winneconne,1,winneconne,Wisconsin,509,wisconsin,Winneconne,Winneconne
9058,15697,8186,Christmas Mountain Village,S944 Christmas Mountain Rd,"Wisconsin Dells, Wisconsin 53965",53965,(608) 254-3971,18,Art Johnson,1985,...,NaN,52,Wisconsin Dells,6,wisconsindells,Wisconsin,509,wisconsin,Wisconsin Dells,Wisconsin Dells
9059,15700,8186,Trappers Turn Golf Club,652 Trappers Turn Dr,"Wisconsin Dells, Wisconsin 53965",53965,(800) 221-8876,18,Andy North,1991,...,NaN,52,Wisconsin Dells,6,wisconsindells,Wisconsin,509,wisconsin,Wisconsin Dells,Wisconsin Dells
9060,15702,8187,Bull's Eye Country Club,1 Airport Ave,"Wisconsin Rapids, Wisconsin 54494",54494,(715) 423-2225,18,Leonard Macomber,1927,...,NaN,52,Wisconsin Rapids,4,wisconsinrapids,Wisconsin,509,wisconsin,Wisconsin Rapids,Wisconsin Rapids


### All available information for the golf kiosk

In [18]:
# Select the golf course columns needed for course longitude, latitude
golf_course_df = MissRiver_golf[["course_id",
                                 "course",
                                 "city",
                                 "state",
                                 "street",
                                 "zip_code",
                                 "lng",
                                 "lat",
                                 "hole",
                                 "public_private",
                                 "golf_season"]]
                     
# Display the new dataframe
golf_course_df

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private,golf_season
386,607,Eagle Crest,Alma,Arkansas,"3926 Golf Course Drive,",72921,NaN,NaN,18,Public,year round
387,611,Millwood Landing Golf & RV Resort,Ashdown,Arkansas,596 Hwy 317,71822,NaN,NaN,18,Private,year round
388,616,Country Club at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,NaN,NaN,18,Private,year round
389,617,Highlands at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,NaN,NaN,18,Private,year round
390,618,Kingswood at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,NaN,NaN,18,Private,year round
...,...,...,...,...,...,...,...,...,...,...,...
9057,15694,Lake Breeze Golf Club,Winneconne,Wisconsin,6333 State Road 110,54986,NaN,NaN,18,Public,Open: 4/01 Closed: 11/01
9058,15697,Christmas Mountain Village,Wisconsin Dells,Wisconsin,S944 Christmas Mountain Rd,53965,NaN,NaN,18,Resort,Open: 4/01 Closed: 11/01
9059,15700,Trappers Turn Golf Club,Wisconsin Dells,Wisconsin,652 Trappers Turn Dr,53965,NaN,NaN,18,Public,Open: 4/01 Closed: 11/01
9060,15702,Bull's Eye Country Club,Wisconsin Rapids,Wisconsin,1 Airport Ave,54494,NaN,NaN,18,Private,Open: 4/01 Closed: 11/01


In [19]:
# Select the golf course columns needed for golf kiosk - general information
course_info = golf_df[["course_id",
                       "phone",
                       "architect",
                       "year_built",
                       "guest_policy",
                       "credit_card"]]
                     
# Display the new dataframe
course_info

,course_id,phone,architect,year_built,guest_policy,credit_card
0,2,(606) 635-2106,NaN,1962,Open,"VISA, MasterCard Welcomed"
1,3,(256) 234-2572,Thomas H. Nicol,1961,Closed,"VISA, MasterCard, Amex Accepted"
2,4,(256) 268-2920,"Robert Trent Jones, Sr.",1972,Open,"VISA, MasterCard, Amex, Discover Accepted"
3,6,(334) 222-5282,Evans Barnes,1927,Reciprocal,"VISA, MasterCard Accepted"
4,9,(256) 237-4615,NaN,1909,Closed,None
...,...,...,...,...,...,...
9079,15740,(307) 352-1415,Lynn Radike,1979,Open,"VISA, MasterCard Welcomed"
9080,15741,(307) 326-5222,Henry Hughes,1962,Closed,None
9081,15743,(307) 672-5323,Dick Bailey,1997,Open,"VISA, MasterCard, Amex, Discover Welcomed"
9082,15744,(307) 674-8148,Edward A. Hunnicutt,1932,Open,"VISA, MasterCard Welcomed"


In [20]:
# Select the golf course columns needed for golf kiosk - course details
course_details = golf_df[["course_id",
                          "range",
                          "rental_club",
                          "pro_in_House",
                          "metal_spikes_okay",
                          "weekday",
                          "weekend",
                          "tee_time_welcomed",
                          "rental_cart_available"]]
                     
# Display the new dataframe
course_details

,course_id,range,rental_club,pro_in_House,metal_spikes_okay,weekday,weekend,tee_time_welcomed,rental_cart_available
0,2,No,Yes,Yes,Yes,$,$,Yes,Yes
1,3,Yes,Yes,Yes,Yes,$,$,Yes,Yes
2,4,Yes,Yes,Yes,No,$,$,Yes,Yes
3,6,Yes,No,Yes,Yes,$,$,No,Yes
4,9,Yes,Yes,Yes,No,$,$,No,Yes
...,...,...,...,...,...,...,...,...,...
9079,15740,Yes,Yes,Yes,No,$,$,Yes,Yes
9080,15741,Yes,Yes,Yes,Yes,$$,$$,No,Yes
9081,15743,Yes,Yes,Yes,No,$,$,Yes,Yes
9082,15744,Yes,Yes,Yes,No,$,$,Yes,Yes


In [21]:
# Select the golf course columns needed for golf kiosk - championship tees
championship_tees = golf_df[["course_id", 
                            "championship_par", 
                            "championship_yards", 
                            "championship_slope", 
                            "championship_usga"]]
                     
# Display the new dataframe
championship_tees

,course_id,championship_par,championship_yards,championship_slope,championship_usga
0,2,71,6219,118,69.30
1,3,72,7084,137,74.60
2,4,72,6900,129,70.90
3,6,72,6356,126,69.90
4,9,70,6254,121,70.20
...,...,...,...,...,...
9079,15740,72,7062,122,72.40
9080,15741,72,7163,123,72.30
9081,15743,72,6934,128,72.30
9082,15744,72,6677,114,70.70


In [22]:
# Select the golf course columns needed for golf kiosk - middle tees
middle_tees = golf_df[["course_id",
                       "middle_par",
                       "middle_yards",
                       "middle_slope",
                       "middle_usga"]]
                     
# Display the new dataframe
middle_tees

,course_id,middle_par,middle_yards,middle_slope,middle_usga
0,2,71,5942,115,67.60
1,3,72,6631,135,72.20
2,4,72,6518,126,69.90
3,6,72,6151,124,69.00
4,9,70,5924,118,68.70
...,...,...,...,...,...
9079,15740,72,6741,118,71.00
9080,15741,72,6649,114,69.80
9081,15743,72,6469,122,70.00
9082,15744,72,6404,111,69.10


In [23]:
# Select the golf course columns needed for golf kiosk - forward tees
forward_tees = golf_df[["course_id",
                        "forward_par",
                        "forward_yards",
                        "forward_slope",
                        "forward_usga"]]
                     
# Display the new dataframe
forward_tees

,course_id,forward_par,forward_yards,forward_slope,forward_usga
0,2,75,5418,118,70.30
1,3,73,5373,120,70.70
2,4,72,5518,120,69.80
3,6,72,5563,119,71.20
4,9,72,5086,109,68.50
...,...,...,...,...,...
9079,15740,73,5616,115,73.10
9080,15741,72,5410,110,67.60
9081,15743,72,5878,122,71.90
9082,15744,72,5180,111,68.00


In [24]:
golf_course_df

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private,golf_season
386,607,Eagle Crest,Alma,Arkansas,"3926 Golf Course Drive,",72921,NaN,NaN,18,Public,year round
387,611,Millwood Landing Golf & RV Resort,Ashdown,Arkansas,596 Hwy 317,71822,NaN,NaN,18,Private,year round
388,616,Country Club at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,NaN,NaN,18,Private,year round
389,617,Highlands at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,NaN,NaN,18,Private,year round
390,618,Kingswood at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,NaN,NaN,18,Private,year round
...,...,...,...,...,...,...,...,...,...,...,...
9057,15694,Lake Breeze Golf Club,Winneconne,Wisconsin,6333 State Road 110,54986,NaN,NaN,18,Public,Open: 4/01 Closed: 11/01
9058,15697,Christmas Mountain Village,Wisconsin Dells,Wisconsin,S944 Christmas Mountain Rd,53965,NaN,NaN,18,Resort,Open: 4/01 Closed: 11/01
9059,15700,Trappers Turn Golf Club,Wisconsin Dells,Wisconsin,652 Trappers Turn Dr,53965,NaN,NaN,18,Public,Open: 4/01 Closed: 11/01
9060,15702,Bull's Eye Country Club,Wisconsin Rapids,Wisconsin,1 Airport Ave,54494,NaN,NaN,18,Private,Open: 4/01 Closed: 11/01


### Getting the longitude and latitude for each golf course based on street address first, otherwise zip code

In [25]:
# Getting the longitude and latitude for each golf course
count = 0

# Use the street, city, state to get lng and lat
for index, row in golf_course_df.iterrows():
    # get street, city, state from golf_course_df
    street = row["street"]
    city = row["city"]
    state = row["state"]
    postal_code = row["zip_code"]
    
    # Getting lat/lng
    query_address_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={street},+{city},+{state}&key={gkey}"
    # print(f"url = {query_address_url}")
        
    query_zip_url = f"https://maps.googleapis.com/maps/api/geocode/json?components=postal_code:{postal_code}&key={gkey}"
    # print(f"url = {query_zip_url}")

    response = requests.get(query_address_url).json()
    # print(f"response = {response}")
    
    count = count + 1
    # print(f"#{count} - index = {index}")
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        golf_course_df.loc[index, "lat"] = response["results"][0]["geometry"]["location"]["lat"]
        golf_course_df.loc[index, "lng"] = response["results"][0]["geometry"]["location"]["lng"]
        
    except (KeyError, IndexError):
        response_zip = requests.get(query_zip_url).json()
        # print(f"response_zip = {response_zip}")golf_course_df.loc[index, "lat"] = response_zip["results"][0]["geometry"]["location"]["lat"]
        golf_course_df.loc[index, "lng"] = response_zip["results"][0]["geometry"]["location"]["lng"]
        print("Using zip code for lng, lat.")
        

C:\Users\Jeff Brown\anaconda3\envs\NewPythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [26]:
# Preview course subset
golf_course_df

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private,golf_season
386,607,Eagle Crest,Alma,Arkansas,"3926 Golf Course Drive,",72921,-94.167763,35.500004,18,Public,year round
387,611,Millwood Landing Golf & RV Resort,Ashdown,Arkansas,596 Hwy 317,71822,-94.011486,33.717971,18,Private,year round
388,616,Country Club at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,year round
389,617,Highlands at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,year round
390,618,Kingswood at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,year round
...,...,...,...,...,...,...,...,...,...,...,...
9057,15694,Lake Breeze Golf Club,Winneconne,Wisconsin,6333 State Road 110,54986,-88.655907,44.119615,18,Public,Open: 4/01 Closed: 11/01
9058,15697,Christmas Mountain Village,Wisconsin Dells,Wisconsin,S944 Christmas Mountain Rd,53965,-89.860045,43.609059,18,Resort,Open: 4/01 Closed: 11/01
9059,15700,Trappers Turn Golf Club,Wisconsin Dells,Wisconsin,652 Trappers Turn Dr,53965,-89.797500,43.633333,18,Public,Open: 4/01 Closed: 11/01
9060,15702,Bull's Eye Country Club,Wisconsin Rapids,Wisconsin,1 Airport Ave,54494,-89.847278,44.365327,18,Private,Open: 4/01 Closed: 11/01


In [27]:
# Create a dataframe to export
golf_courses = golf_course_df[["course_id",
                               "course",
                               "city",
                               "state",
                               "street",
                               "zip_code",
                               "lng",
                               "lat",
                               "hole",
                               "public_private", 
                               "golf_season"]]
                     
# Display the new dataframe
golf_courses

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private,golf_season
386,607,Eagle Crest,Alma,Arkansas,"3926 Golf Course Drive,",72921,-94.167763,35.500004,18,Public,year round
387,611,Millwood Landing Golf & RV Resort,Ashdown,Arkansas,596 Hwy 317,71822,-94.011486,33.717971,18,Private,year round
388,616,Country Club at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,year round
389,617,Highlands at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,year round
390,618,Kingswood at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,year round
...,...,...,...,...,...,...,...,...,...,...,...
9057,15694,Lake Breeze Golf Club,Winneconne,Wisconsin,6333 State Road 110,54986,-88.655907,44.119615,18,Public,Open: 4/01 Closed: 11/01
9058,15697,Christmas Mountain Village,Wisconsin Dells,Wisconsin,S944 Christmas Mountain Rd,53965,-89.860045,43.609059,18,Resort,Open: 4/01 Closed: 11/01
9059,15700,Trappers Turn Golf Club,Wisconsin Dells,Wisconsin,652 Trappers Turn Dr,53965,-89.797500,43.633333,18,Public,Open: 4/01 Closed: 11/01
9060,15702,Bull's Eye Country Club,Wisconsin Rapids,Wisconsin,1 Airport Ave,54494,-89.847278,44.365327,18,Private,Open: 4/01 Closed: 11/01


### Stripping out the beginning and ending months of the golf season for each course

In [28]:
# Golf season begin and end months
for index, row in golf_courses.iterrows():
    str_open = row["golf_season"][0:5]
    beg_mnth = row["golf_season"][6:7]
    str_closed = row["golf_season"][11:18]
    end_mnth = row["golf_season"][19:21]
    end_day = row["golf_season"][22:24]

    if str_closed == " Closed" :
        beg_mnth = row["golf_courses"][6:8]
        str_closed = row["golf_courses"][12:19]
        end_mnth = row["golf_courses"][20:22]
        end_day = row["golf_courses"][22:24]
   
    if str_closed == "Closed:" :
        if end_day == "01":
            if end_mnth == "1/":
                end_mnth = "12"
            elif end_mnth == "4/":
                end_mnth = "3"
            elif end_mnth == "5/":
                end_mnth = "4"
            elif end_mnth == "6/":
                end_mnth = "5"
            elif end_mnth == "7/":
                end_mnth = "6"
            elif end_mnth == "8/":
                end_mnth = "7"
            elif end_mnth == "9/":
                end_mnth = "7"          
            elif end_mnth == "10":
                end_mnth = "9"
            elif end_mnth == "11":
                end_mnth = "10"             
            elif end_mnth == "12":
                end_mnth = "11"
        else:
            if end_mnth == "1/":
                end_mnth = "01"
            elif end_mnth == "4/":
                end_mnth = "04"
            elif end_mnth == "5/":
                end_mnth = "05"
            elif end_mnth == "6/":
                end_mnth = "06"
            elif end_mnth == "7/":
                end_mnth = "07"
            elif end_mnth == "8/":
                end_mnth = "08"
            elif end_mnth == "9/":
                end_mnth = "09"          
    else:
        end_mnth = "12"

   
    if str_open == "Open:" :
        if beg_mnth not in ["2", "3", "4", "5", "6", "7", "10", "11", "12"]:
            beg_mnth = "1"
    else:
        beg_mnth = "1"      

    golf_courses.loc[index,"beg_mnth"] = beg_mnth
    golf_courses.loc[index,"end_mnth"] = end_mnth
    
golf_courses

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private,golf_season,beg_mnth,end_mnth
386,607,Eagle Crest,Alma,Arkansas,"3926 Golf Course Drive,",72921,-94.167763,35.500004,18,Public,year round,1,12
387,611,Millwood Landing Golf & RV Resort,Ashdown,Arkansas,596 Hwy 317,71822,-94.011486,33.717971,18,Private,year round,1,12
388,616,Country Club at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,year round,1,12
389,617,Highlands at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,year round,1,12
390,618,Kingswood at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,year round,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9057,15694,Lake Breeze Golf Club,Winneconne,Wisconsin,6333 State Road 110,54986,-88.655907,44.119615,18,Public,Open: 4/01 Closed: 11/01,4,10
9058,15697,Christmas Mountain Village,Wisconsin Dells,Wisconsin,S944 Christmas Mountain Rd,53965,-89.860045,43.609059,18,Resort,Open: 4/01 Closed: 11/01,4,10
9059,15700,Trappers Turn Golf Club,Wisconsin Dells,Wisconsin,652 Trappers Turn Dr,53965,-89.797500,43.633333,18,Public,Open: 4/01 Closed: 11/01,4,10
9060,15702,Bull's Eye Country Club,Wisconsin Rapids,Wisconsin,1 Airport Ave,54494,-89.847278,44.365327,18,Private,Open: 4/01 Closed: 11/01,4,10


### Bringing all the information for the golf kiosk together

In [29]:
# Merge the tee information with the golf courses dataframe using a left join
golf_tees1 = pd.merge(golf_courses, championship_tees, on="course_id" ,how="left")
golf_tees2 = pd.merge(golf_tees1, middle_tees, on="course_id" ,how="left")
golf_tees3 = pd.merge(golf_tees2, forward_tees, on="course_id" ,how="left")
golf_tees3

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private,...,championship_slope,championship_usga,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga
0,607,Eagle Crest,Alma,Arkansas,"3926 Golf Course Drive,",72921,-94.167763,35.500004,18,Public,...,125,73.30,71,6246,120,70.30,71,5257,117,70.40
1,611,Millwood Landing Golf & RV Resort,Ashdown,Arkansas,596 Hwy 317,71822,-94.011486,33.717971,18,Private,...,118,70.40,72,6159,116,69.30,72,4731,103,62.90
2,616,Country Club at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,...,126,73.00,72,6432,122,70.80,72,5537,116,71.60
3,617,Highlands at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,...,135,72.90,72,6098,127,68.90,72,5332,116,69.70
4,618,Kingswood at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,...,113,69.40,72,5922,110,67.80,71,5103,110,68.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698,15694,Lake Breeze Golf Club,Winneconne,Wisconsin,6333 State Road 110,54986,-88.655907,44.119615,18,Public,...,121,72.20,72,6378,116,69.90,72,5748,118,71.90
1699,15697,Christmas Mountain Village,Wisconsin Dells,Wisconsin,S944 Christmas Mountain Rd,53965,-89.860045,43.609059,18,Resort,...,128,71.70,71,6179,125,70.30,71,5212,123,70.00
1700,15700,Trappers Turn Golf Club,Wisconsin Dells,Wisconsin,652 Trappers Turn Dr,53965,-89.797500,43.633333,18,Public,...,125,70.10,72,5842,122,68.50,72,5017,119,69.30
1701,15702,Bull's Eye Country Club,Wisconsin Rapids,Wisconsin,1 Airport Ave,54494,-89.847278,44.365327,18,Private,...,131,72.30,72,6356,128,71.30,73,5741,129,73.60


In [30]:
# Export the golf course data to csv
golf_tees3.to_csv("results/MissRiver_golf.csv", index=False, header=True)

In [31]:
# Merge more details with the golf courses dataframe using a left join
golf_info = pd.merge(golf_tees3, course_info, on="course_id" ,how="left")
golf_info

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private,...,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,phone,architect,year_built,guest_policy,credit_card
0,607,Eagle Crest,Alma,Arkansas,"3926 Golf Course Drive,",72921,-94.167763,35.500004,18,Public,...,70.30,71,5257,117,70.40,(479) 632-8857,Mark Hayes,1996,Open,"VISA, MasterCard, Amex Welcomed"
1,611,Millwood Landing Golf & RV Resort,Ashdown,Arkansas,596 Hwy 317,71822,-94.011486,33.717971,18,Private,...,69.30,72,4731,103,62.90,(870) 898-6674,NaN,1975,Closed,"VISA, MasterCard Welcomed"
2,616,Country Club at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,...,70.80,72,5537,116,71.60,(501) 855-8172,Joe Finger,1968,Closed,"VISA, MasterCard, Discover Welcomed"
3,617,Highlands at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,...,68.90,72,5332,116,69.70,(501) 855-8172,"Ault, Clark & Associates",1988,Closed,"VISA, MasterCard, Discover Welcomed"
4,618,Kingswood at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,...,67.80,71,5103,110,68.70,(501) 855-8172,"Ault, Clark & Associates",1971,Closed,"VISA, MasterCard, Discover Welcomed"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698,15694,Lake Breeze Golf Club,Winneconne,Wisconsin,6333 State Road 110,54986,-88.655907,44.119615,18,Public,...,69.90,72,5748,118,71.90,(920) 582-7585,Homer Fieldhouse,1990,Open,"VISA, MasterCard Welcomed"
1699,15697,Christmas Mountain Village,Wisconsin Dells,Wisconsin,S944 Christmas Mountain Rd,53965,-89.860045,43.609059,18,Resort,...,70.30,71,5212,123,70.00,(608) 254-3971,Art Johnson,1985,Open,"VISA, MasterCard, Amex, Discover Welcomed"
1700,15700,Trappers Turn Golf Club,Wisconsin Dells,Wisconsin,652 Trappers Turn Dr,53965,-89.797500,43.633333,18,Public,...,68.50,72,5017,119,69.30,(800) 221-8876,Andy North,1991,Open,"VISA, MasterCard, Amex, Discover Welcomed"
1701,15702,Bull's Eye Country Club,Wisconsin Rapids,Wisconsin,1 Airport Ave,54494,-89.847278,44.365327,18,Private,...,71.30,73,5741,129,73.60,(715) 423-2225,Leonard Macomber,1927,Reciprocal,None


In [32]:
# Merge more details with the golf courses dataframe using a left join
golf_details = pd.merge(golf_info, course_details, on="course_id" ,how="left")
golf_details

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private,...,guest_policy,credit_card,range,rental_club,pro_in_House,metal_spikes_okay,weekday,weekend,tee_time_welcomed,rental_cart_available
0,607,Eagle Crest,Alma,Arkansas,"3926 Golf Course Drive,",72921,-94.167763,35.500004,18,Public,...,Open,"VISA, MasterCard, Amex Welcomed",Yes,Yes,Yes,No,33$,38$,Yes,Included in fee
1,611,Millwood Landing Golf & RV Resort,Ashdown,Arkansas,596 Hwy 317,71822,-94.011486,33.717971,18,Private,...,Closed,"VISA, MasterCard Welcomed",Yes,Yes,No,No,$,$,No,Yes
2,616,Country Club at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,...,Closed,"VISA, MasterCard, Discover Welcomed",Yes,Yes,Yes,No,$,$,Yes,Yes
3,617,Highlands at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,...,Closed,"VISA, MasterCard, Discover Welcomed",Yes,Yes,Yes,No,$,$,Yes,Yes
4,618,Kingswood at Bella Vista Village Golf Courses,Bella Vista,Arkansas,101 Town Center,72714,-94.249018,36.475194,18,Private,...,Closed,"VISA, MasterCard, Discover Welcomed",Yes,Yes,Yes,No,$,$,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698,15694,Lake Breeze Golf Club,Winneconne,Wisconsin,6333 State Road 110,54986,-88.655907,44.119615,18,Public,...,Open,"VISA, MasterCard Welcomed",Yes,Yes,Yes,No,$,$,Yes,Yes
1699,15697,Christmas Mountain Village,Wisconsin Dells,Wisconsin,S944 Christmas Mountain Rd,53965,-89.860045,43.609059,18,Resort,...,Open,"VISA, MasterCard, Amex, Discover Welcomed",Yes,Yes,Yes,No,$,$,Yes,Yes
1700,15700,Trappers Turn Golf Club,Wisconsin Dells,Wisconsin,652 Trappers Turn Dr,53965,-89.797500,43.633333,18,Public,...,Open,"VISA, MasterCard, Amex, Discover Welcomed",Yes,Yes,Yes,No,$$,$$,Yes,Included in fee
1701,15702,Bull's Eye Country Club,Wisconsin Rapids,Wisconsin,1 Airport Ave,54494,-89.847278,44.365327,18,Private,...,Reciprocal,None,Yes,Yes,Yes,No,$$,$$,Yes,Yes


In [33]:
# Export the golf course data to csv
golf_details.to_csv("results/MissRiver_golf_details.csv", index=False, header=True)

## Step 2:  To load the golf details into a PostgreSQL database, run golf_db_load.ipynb

Before running the load process, follow these steps:

1. create a new database in ProstgreSQL and name it Golf

2. run schema.sql to create the table in the new database

3. use a config.py file to store username and password for the new database you just created - it should look something like this

        username = "postgres"
        password = "mypassword"
